In [1]:
import os
import sys
import inspect
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

pathname = inspect.getframeinfo(inspect.currentframe()).filename
sys.path.append(os.path.abspath(os.path.dirname(pathname))+'/classes/')

from Model import *
import radiativeTransfer

In [2]:
complete = False

x = 36
y = 36
z = 2

shape = 'disk'

resolution = 1000

modelFolder = 'r1000.0_n3015'

selection = 0

In [3]:
print('KOSMA-tau^3')

species = ['13CO 10', 'C+ 1', 'CO 1', 'CO 2', 'CO 3', 'CO 4', 'CO 5', 'CO 6', 'CO 7', 'CO 8', 'CO 9', 'CO 10', '13CO 1', '13CO 2', '13CO 3', '13CO 4', '13CO 5', '13CO 6', '13CO 7', '13CO 8', '13CO 9', '13CO 10', 'O 2']
kosma = Model(x, y, z, modelName='MilkyWay', modelType=shape, resolution=resolution)

KOSMA-tau^3
<built-in method max of numpy.ndarray object at 0x7fea5c4b9850>


In [4]:
kosma.addSpecies(species)
kosma.calculateModel()
kosma.writeEmission()
radiativeTransfer.calculateObservation(directory=constants.HISTORYPATH+constants.directory+modelFolder)

TypeError: 'NoneType' object is not subscriptable

In [5]:
import cygrid

ImportError: /home/craig/.local/lib/python3.6/site-packages/cygrid/cygrid.cpython-36m-x86_64-linux-gnu.so: undefined symbol: __kmpc_reduce_nowait